# AB Tests with Python "free trial" Screener
**Experiment Name**: "Free Trial" Screener.

In [ ]:
import math as mt
import numpy as np
import pandas as pd
from scipy.stats import norm

## Current Setup

- Udacity course pages currently show two options: **“Start free trial”** and **“Access course materials.”**
  - **Start free trial:** Students enter credit card info and get a 14-day trial of the paid version. They’re charged automatically after 14 days unless they cancel.  
  - **Access course materials:** Students can view videos and take quizzes for free, but don’t get coaching, feedback, or a certificate.

## The Experiment

- Udacity tested adding a **time commitment question** after clicking “Start free trial.”  
  - If a student said they could spend **5+ hours/week**, they continued to checkout as usual.  
  - If they said **less than 5 hours/week**, they saw a message explaining that Udacity courses usually need more time and suggesting they might prefer the free materials option.  
  - Students could then **either continue with the free trial** or **choose the free materials**.

## Hypothesis

The change would help set realistic expectations and reduce cancellations from students who didn’t have enough time—without significantly lowering the number of paying or completing students.  
This could improve student satisfaction and let coaches focus on learners likely to finish.

## Experiment Details

- **Unit of diversion:** cookie  
- If a student enrolls in a free trial, tracking switches to **user ID** (a user can’t start multiple free trials).  
- Users who don’t enroll aren’t tracked by user ID, even if they were signed in.


## Metric Choice

A successful experiment needs two types of metrics: **Invariant** and **Evaluation** metrics.

- **Invariant metrics**: Should *not* be affected by the experiment. They help verify that randomization worked and groups are comparable.
- **Evaluation metrics**: Measure the impact of the experiment and relate directly to business goals.

Each metric includes a **$D_{min}$** — the minimum meaningful change for the business.  
(Example: a retention increase below 2% may be statistically significant but not practically useful.)

### Invariant Metrics – Sanity Checks

| Metric Name | Formula | $D_{min}$ | Notation |
|--------------|----------|------------|-----------|
| Cookies on course overview page | # unique daily cookies on page | 3000 cookies | $C_k$ |
| Clicks on Free Trial button | # unique daily cookies who clicked | 240 clicks | $C_l$ |
| Free Trial Click-Through Probability | $\frac{C_l}{C_k}$ | 0.01 | $CTP$ |

### Evaluation Metrics – Performance Indicators

| Metric Name | Formula | $D_{min}$ | Notation |
|--------------|----------|------------|-----------|
| **Gross Conversion** | $\frac{enrolled}{C_l}$ | 0.01 | $Conversion_{Gross}$ |
| **Retention** | $\frac{paid}{enrolled}$ | 0.01 | $Retention$ |
| **Net Conversion** | $\frac{paid}{C_l}$ | 0.0075 | $Conversion_{Net}$ |


## Estimating the baseline values of metrics 

Before we start our experiment we should know how these metrics behave before the change - that is, what are their baseline values.

### Collecting estimators data 
|Item|Description|Estimator|
|--|--|--|
|Number of cookies|	Daily unique cookies to view course overview page|	40,000|
|Number of clicks|	Daily unique cookies to click Free Trial button	|3,200|
|Number of enrollments	|Free Trial enrollments per day|	660|
|CTP	|CTP on Free Trial button	|0.08|
|Gross Conversion	|Probability of enrolling, given a click	|0.20625|
|Retention	|Probability of payment, given enrollment	|0.53|
|Net Conversion	|Probability of payment, given click	|0.109313|

In [ ]:
# Let's place this estimators into a dictionary for ease of use later
baseline = {"Cookies": 40000,
            "Clicks": 3200,
            "Enrollments": 660,
            "CTP": 0.08,
            "GConversion": 0.20625,
            "Retention": 0.53,
            "NConversion": 0.109313}

### Estimating Standard Deviation

After collecting metric estimates, we calculate the **standard deviation** to use in **sample size** and **confidence interval** calculations.  
A higher variance means it’s harder to detect a statistically significant effect.

Assuming **5,000 cookies** visit the course overview page per day (as stated in the project instructions), we’ll estimate the standard deviation for the **evaluation metrics only**.  
This sample size is smaller than the total population but large enough to form two comparison groups.

### Scaling Collected Data

Before calculating variance, we need to **scale** our collected metric counts to match the sample size used for variance estimation.  
In this case, we scale from **40,000 unique cookies per day** (original data) down to **5,000 cookies per day**.


In [ ]:
#Scale The counts estimates
baseline["Cookies"] = 5000
baseline["Clicks"]=baseline["Clicks"]*(5000/40000)
baseline["Enrollments"]=baseline["Enrollments"]*(5000/40000)
baseline

### Estimating Analytically

To estimate variance analytically, we assume metrics that represent probabilities $(\hat{p})$ follow a **binomial distribution**.  
The standard deviation can then be calculated using:

$$
SD = \sqrt{\frac{\hat{p}(1-\hat{p})}{n}}
$$

This assumption holds only when the **unit of diversion** (how users are split) matches the **unit of analysis** (the denominator in the metric formula).  

### What Happens if Units Differ?

Suppose the **unit of diversion** is a user ID, but a single user may click multiple times:

- User A clicks 3 times → potentially 3 “trials” in the denominator.  
- User B clicks once → 1 trial.  

Outcomes from the same user are **correlated** (if User A doesn’t enroll once, they probably won’t enroll on other clicks).

Mathematically:

- **Independent Bernoulli trials:**  
$
Var(X) = n p (1-p)
$

- **Correlated trials:**  
$
Var(X) > n p (1-p)
$ (assuming positive correlation between all users)

**Why?** Because repeated measures from the same user aren’t adding as much independent information. The effective sample size is smaller than the count of trials. The actual variance may vary, and it’s better to estimate it **empirically** using collected data.

**Gross Conversion** – The baseline represents the probability of enrollment given a click.

> In this case, the **unit of diversion** (cookies) — how users are split between control and experiment — is the same as the **unit of analysis** (cookies who click), which is the denominator in the Gross Conversion formula.  

Because these units match, the **analytical estimate of variance** is valid and sufficient.


In [ ]:
# Let's get the p and n we need for Gross Conversion (GC)
# and compute the Stansard Deviation(sd) rounded to 4 decimal digits.
GC={}
GC["d_min"]=0.01
GC["p"]=baseline["GConversion"]
#p is given in this case - or we could calculate it from enrollments/clicks
GC["n"]=baseline["Clicks"]
GC["sd"]=round(mt.sqrt((GC["p"]*(1-GC["p"]))/GC["n"]),4)
GC

**Retention** - The baseline is the probability of payment, given enrollment. The sample size is the number of enrolled users. 

>In this case, unit of diversion is not equal to unit of analysis (users who enrolled) so an analytical estimation is not enough 

If we had the data for these estimates, we would want to estimate this variance empirically as well.

In [ ]:
# Let's get the p and n we need for Retention(R)
# and compute the Stansard Deviation(sd) rounded to 4 decimal digits.
R={}
R["d_min"]=0.01
R["p"]=baseline["Retention"]
R["n"]=baseline["Enrollments"]
R["sd"]=round(mt.sqrt((R["p"]*(1-R["p"]))/R["n"]),4)
R

**Net Conversion** - The baseline is the probability of payment, given a click. The sample size is the number of cookies that clicked. 
> In this case, the unit of analysis and diversion are equal so we expect a good enough estimation analytically.


In [ ]:
# Let's get the p and n we need for Net Conversion (NC)
# and compute the Standard Deviation (sd) rounded to 4 decimal digits.
NC={}
NC["d_min"]=0.0075
NC["p"]=baseline["NConversion"]
NC["n"]=baseline["Clicks"]
NC["sd"]=round(mt.sqrt((NC["p"]*(1-NC["p"]))/NC["n"]),4)
NC

# Experiment Sizing

Given  $\alpha = 0.05$ (significance level) and  $\beta = 0.2$ (power), we want to estimate how many total pageviews (cookies who viewed the course overview page) are needed in the experiment. This total will be divided into the two groups: control and experiment.

The minimum sample size for control and experiment groups, which provides a probability of **Type I Error** $\alpha$, **Power** $1−\beta$, **detectable effect** $d$, and **baseline conversion rate** $p$ (simple hypothesis)  

$$
H_0: P_{cont} - P_{exp} = 0
$$

against the simple alternative  

$$
H_A: P_{cont} - P_{exp} = d
$$

is:

$$
n = \frac{\left(Z_{1-\frac{\alpha}{2}} sd_1 + Z_{1-\beta} sd_2 \right)^2}{d^2}
$$

where

$$
sd_1 = \sqrt{2 p (1-p)}, \quad sd_2 = \sqrt{p (1-p) + (p+d)(1-(p+d))}
$$


# Sample Size Formula for Two-Sample Proportions

Suppose we want to compare two proportions:

- Control group proportion: $p_1 = p$  
- Experiment group proportion: $p_2 = p + d$  
- Detectable difference: $d = p_2 - p_1$  

We aim for:

- Significance level: $\alpha$ (Type I error)  
- Power: $1 - \beta$ (probability of detecting a true effect)  

---

## 1. Test Statistic

For a two-sided z-test comparing proportions, the test statistic is:

$$
Z = \frac{\hat{p}_1 - \hat{p}_2}{\sqrt{\frac{p_1 (1-p_1)}{n} + \frac{p_2 (1-p_2)}{n}}}
$$

where $n$ is the sample size per group.  

- **Under $H_0$**: $p_1 = p_2 = p$

$$
SD_0 = \sqrt{\frac{p(1-p)}{n} + \frac{p(1-p)}{n}} = \sqrt{\frac{2 p(1-p)}{n}}
$$

- **Under $H_A$**: $p_1 = p, p_2 = p+d$

$$
SD_A = \sqrt{\frac{p(1-p) + (p+d)(1-(p+d))}{n}}
$$

---

## 2. Critical Values for Significance and Power

- Two-sided test significance $\alpha$:

$$
Z_{\text{crit}} = Z_{1-\frac{\alpha}{2}}
$$

- To achieve power $1-\beta$, we require:

$$
P(\text{Reject } H_0 \mid H_A) = 1 - \beta
$$

---

## 3. Relating Detectable Difference to z-Scores

Let $X = \hat{p}_1 - \hat{p}_2$. Under $H_A$, 

$$
X \sim N(d, SD_A^2)
$$

Standardizing:

$$
P\left( \frac{X - d}{SD_A} > \frac{Z_{\text{crit}} SD_0 - d}{SD_A} \right) = 1 - \beta
$$

By the definition of the standard normal quantile:

$$
\frac{Z_{\text{crit}} SD_0 - d}{SD_A} = -Z_{1-\beta}
$$

Rearranging gives:

$$
d = Z_{1-\frac{\alpha}{2}} SD_0 + Z_{1-\beta} SD_A
$$

Substitute $SD_0$ and $SD_A$:

$$
d = Z_{1-\frac{\alpha}{2}} \sqrt{\frac{2 p(1-p)}{n}} + Z_{1-\beta} \sqrt{\frac{p(1-p) + (p+d)(1-(p+d))}{n}}
$$

Factor out $1/\sqrt{n}$:

$$
d = \frac{1}{\sqrt{n}} \Bigg( Z_{1-\frac{\alpha}{2}} \sqrt{2 p(1-p)} + Z_{1-\beta} \sqrt{p(1-p) + (p+d)(1-(p+d))} \Bigg)
$$

---

## 4. Solve for Sample Size $n$

$$
\sqrt{n} = \frac{Z_{1-\frac{\alpha}{2}} \sqrt{2 p(1-p)} + Z_{1-\beta} \sqrt{p(1-p) + (p+d)(1-(p+d))}}{d}
$$

$$
\boxed{
n = \frac{\Big( Z_{1-\frac{\alpha}{2}} \sqrt{2 p(1-p)} + Z_{1-\beta} \sqrt{p(1-p) + (p+d)(1-(p+d))} \Big)^2}{d^2}
}
$$

This gives the **required sample size per group** to detect a difference $d$ with significance level $\alpha$ and power $1-\beta$.


Regarding inputs, we have all the data we need: Type 1 error $(\alpha)$, power $(1−\beta)$, detectable change $(d=Dmin)$ and baseline conversion rate, our $\hat{p}$. What we need to calculate:

- Get Z score for $1−\frac{α}{2}$ and for $1−\beta$
 
- Get standard deviations 1 & 2, that is for both the baseline and for expected changed rate.

In [ ]:
#Inputs: required alpha value (alpha should already fit the required test)
#Returns: z-score for given alpha
def get_z_score(alpha):
    return norm.ppf(alpha)

# Inputs p-baseline conversion rate which is our estimated p and d-minimum detectable change
# Returns
def get_sds(p,d):
    sd1=mt.sqrt(2*p*(1-p))
    sd2=mt.sqrt(p*(1-p)+(p+d)*(1-(p+d)))
    sds=[sd1,sd2]
    return sds

# Inputs:sd1-sd for the baseline,sd2-sd for the expected change,alpha,beta,d-d_min,p-baseline estimate p
# Returns: the minimum sample size required per group according to metric denominator
def get_sampSize(sds,alpha,beta,d):
    n=pow((get_z_score(1-alpha/2)*sds[0]+get_z_score(1-beta)*sds[1]),2)/pow(d,2)
    return n

## Calculate Sample Size per Metric

### Gross Conversion


In [ ]:
GC["d"]=0.01
R["d"]=0.01
NC["d"]=0.0075

In [ ]:
# Let's get an integer value for simplicity
GC["SampSize"]=round(get_sampSize(get_sds(GC["p"],GC["d"]),0.05,0.2,GC["d"]))
GC["SampSize"]

This means we need at least 25,835 cookies who click the Free Trial button - per group! That means that if we got 400 clicks out of 5000 pageviews (400/5000 = 0.08) -> So, we are going to need GC["SampSize"]/0.08 = 322,938 pageviews, again ; per group! Finally, the total amount of samples per the Gross Conversion metric is:

In [ ]:
GC["SampSize"]=round(GC["SampSize"]/0.08*2)
GC["SampSize"]

### Retention

In [ ]:
# Getting a nice integer value
R["SampSize"]=round(get_sampSize(get_sds(R["p"],R["d"]),0.05,0.2,R["d"]))
R["SampSize"]

This means that we need 39,087 users who enrolled per group! We have to first convert this to cookies who clicked, and then to cookies who viewed the page, then finally to multipky by two for both groups.

In [ ]:
R["SampSize"]=round(R["SampSize"]/0.08/0.20625*2)
R["SampSize"]

This takes us as high as over 4 million page views total, this is practically impossible because we know we get about 40,000 a day, this would take well over 100 days. This means we have to drop this metric and not continue to work with it because results from our experiment (which is much smaller) will be biased.

### Net Conversion

In [ ]:
# Getting a nice integer value
NC["SampSize"]=round(get_sampSize(get_sds(NC["p"],NC["d"]),0.05,0.2,NC["d"]))
NC["SampSize"]

So, needing 27,413 cookies who click per group takes us all the way up to:

In [ ]:
NC["SampSize"]=NC["SampSize"]/0.08*2
NC["SampSize"]

We are all the way up to 685,325 cookies who view the page. This is more than what was needed for Gross Conversion, so this will be our number. Assuming we take 80% of each days pageviews, the data collection period for this experiment (the period in which the experiment is revealed) will be about 3 weeks.

# Analyzing Collected Data

In [ ]:
# we use pandas to load datasets
control=pd.read_csv("control_data.csv")
experiment=pd.read_csv("experiment_data.csv")
control.head()

### Sanity Checks

We have 3 Invariant metrics::

- Number of Cookies in Course Overview Page
- Number of Clicks on Free Trial Button
- Free Trial button Click-Through-Probability (CTP)


In [ ]:
# A significant difference will imply a biased experiment 
# that we should not rely on it's results.

pageviews_cont=control['Pageviews'].sum()
pageviews_exp=experiment['Pageviews'].sum()
pageviews_total=pageviews_cont+pageviews_exp
print ("number of pageviews in control:", pageviews_cont)
print ("number of Pageviewsin experiment:" ,pageviews_exp)

Ok so these look like pretty close numbers. Now, let's make sure this difference in amounts is not significant and is random and even like we expected. We can model this diversion in the following way:
We expect the amount of pageviews in the control group to be about a half (50%) of the total pageviews in both groups, so we can define a random variable with an easy to use distribution.

A binomial random variable will be the number of successes we can expect to get out of N experiments, given the probability of a single success. So, if we consider being assigned to a group (control, for example) a success with probability 0.5 (random!), the number of samples which get assigned to the group is the value of our random binomial variable!

This get's easier thanks to the central limit theorem which let's us approximate the binomial distribution to a normal distribution (when n is large enough) with a mean of  $p$ and a standard deviation $\sqrt{\frac{p(1-p)}{N}}$

$$
X \sim N\left(p, \sqrt{\frac{p(1-p)}{N}}\right)
$$

What we want to test is whether our observed $\hat{p}$ (number of samples in control divided by total number of damples in both groups) is not significantly different than $p=0.5$. 

In order to do that we can calculate the margin of error acceptable at a 95% confidence level:

$$
ME = Z_{1-\frac{\alpha}{2}}SD
$$
with confidence interval
$$
CI = [\hat{p}-ME,\hat{p}+ME]
$$

In [ ]:
p=0.5
alpha=0.05
p_hat=round(pageviews_cont/(pageviews_total),4)
sd=mt.sqrt(p*(1-p)/(pageviews_total))
ME=round(get_z_score(1-(alpha/2))*sd,4)
print ("The confidence interval is between",p-ME,"and",p+ME,"; Is",p_hat,"inside this range?")

Our observed  $\hat{p}$ is inside this range which means the difference in number of samples between groups is expected. So far so good, since this invariant metric sanity test passes!

In [ ]:
# Number of cookies who clicked the Free Trial Button 

clicks_cont=control['Clicks'].sum()
clicks_exp=experiment['Clicks'].sum()
clicks_total=clicks_cont+clicks_exp

p_hat=round(clicks_cont/clicks_total,4)
sd=mt.sqrt(p*(1-p)/clicks_total)
ME=round(get_z_score(1-(alpha/2))*sd,4)
print ("The confidence interval is between",p-ME,"and",p+ME,"; Is",p_hat,"inside this range?")

We have another pass! Great, so far it still seems all is well with our experiment results. Now, for the final metric which is a probability.

### Sanity Checks for differences between probabilities 

**Click-through-probability of the Free Trial Button** In this case, we want to make sure the proportion of clicks given a pageview (our observed CTP) is about the same in both groups. In order to check this out we will calculate the CTP in each group and calculate a confidence interval for the expected difference between them.

I.e. we expect to see no difference $(CTP_{exp}−CTP_{cont}=0)$ with an acceptable margin of error. The changes we should notice are for the calculation of the standard error - which in this case is a pooled standard error.

$$
SD_{pool} = \sqrt{\hat{p}_{pool}(1-\hat{p}_{pool})\left(\frac{1}{N_{cont}}+\frac{1}{N_{exp}}\right)}
$$
with
$$
\hat{p}_{pool} = \frac{x_{cont}+x_{exp}}{N_{cont}+N_{exp}}
$$

In [ ]:
ctp_cont=clicks_cont/pageviews_cont
ctp_exp=clicks_exp/pageviews_exp
d_hat=round(ctp_exp-ctp_cont,4)
p_pooled=clicks_total/pageviews_total
sd_pooled=mt.sqrt(p_pooled*(1-p_pooled)*(1/pageviews_cont+1/pageviews_exp))
ME=round(get_z_score(1-(alpha/2))*sd_pooled,4)
print ("The confidence interval is between",0-ME,"and",0+ME,"; Is",d_hat,"within this range?")

Wonderful. It seems this test has passed with flying colors as well.

### Examining effect size

The next step is looking at the changes between the control and experiment groups with regard to our evaluation metrics to make sure the difference is there, that it is statistically significant and most importantly practically significant (the difference is "big" enough to make the experimented change beneficial to the company).

> Note:  A metric is statistically significant if the confidence interval does not include 0 (that is, you can be confident there was a change), and it is practically significant if the confidence interval does not include the practical significance boundary (that is, you can be confident there is a change that matters to the business.)

In [ ]:
# Gross Conversion 

# Count the total clicks from complete records only
clicks_cont=control["Clicks"].loc[control["Enrollments"].notnull()].sum()
clicks_exp=experiment["Clicks"].loc[experiment["Enrollments"].notnull()].sum()

In [ ]:
#Gross Conversion - number of enrollments divided by number of clicks
enrollments_cont=control["Enrollments"].sum()
enrollments_exp=experiment["Enrollments"].sum()

GC_cont=enrollments_cont/clicks_cont
GC_exp=enrollments_exp/clicks_exp
GC_pooled=(enrollments_cont+enrollments_exp)/(clicks_cont+clicks_exp)
GC_sd_pooled=mt.sqrt(GC_pooled*(1-GC_pooled)*(1/clicks_cont+1/clicks_exp))
GC_ME=round(get_z_score(1-alpha/2)*GC_sd_pooled,4)
GC_diff=round(GC_exp-GC_cont,4)
print("The change due to the experiment is",GC_diff*100,"%")
print("Confidence Interval: [",GC_diff-GC_ME,",",GC_diff+GC_ME,"]")
print ("The change is statistically significant if the CI doesn't include 0.")
print ("In that case, it is practically significant if",-GC["d_min"],"is not in the CI as well.")


According to this result there was a change due to the experiment, that change was both statistically and practically significant. We have a negative change of 2.06%, when we were willing to accept any change greater than 1%. This means the Gross Conversion rate of the experiment group (the one exposed to the change, i.e. asked how many hours they can devote to studying) has decreased as expected by 2% and this change was significant. This means less people enrolled in the Free Trial after due to the pop-up.

In [ ]:
# Net Conversion

#Net Conversion - number of payments divided by number of clicks
payments_cont=control["Payments"].sum()
payments_exp=experiment["Payments"].sum()

NC_cont=payments_cont/clicks_cont
NC_exp=payments_exp/clicks_exp
NC_pooled=(payments_cont+payments_exp)/(clicks_cont+clicks_exp)
NC_sd_pooled=mt.sqrt(NC_pooled*(1-NC_pooled)*(1/clicks_cont+1/clicks_exp))
NC_ME=round(get_z_score(1-alpha/2)*NC_sd_pooled,4)
NC_diff=round(NC_exp-NC_cont,4)
print("The change due to the experiment is",NC_diff*100,"%")
print("Confidence Interval: [",NC_diff-NC_ME,",",NC_diff+NC_ME,"]")
print ("The change is statistically significant if the CI doesn't include 0.")
print ("In that case, it is practically significant if",NC["d_min"],"is not in the CI as well.")

In this case we got a change size of less than a 0.5%, a very small decrease which is not statistically significant, and as such not practically significant.

## Double check with Sign Tests 

In a sign test we get another angle at analyzing the results we got - we check if the trend of change we observed (increase or decrease) was evident in the daily data.

In [ ]:
# let's first create the dataset we need for this:
# start by merging the two datasets

full=control.join(other=experiment,how="inner",lsuffix="_cont",rsuffix="_exp")
full.head()

In [ ]:
full.count()

In [ ]:
# now we only need the complete data records
full=full.loc[full["Enrollments_cont"].notnull()]
full.count()

In [ ]:
# Perfect! Now, derive a new column for each metric, so we have it's daily values

# We need a 1 if the experiment value is greater than the control value=
x=full['Enrollments_cont']/full['Clicks_cont']
y=full['Enrollments_exp']/full['Clicks_exp']
full['GC'] = np.where(x<y,1,0)

# The same now for net conversion
z=full['Payments_cont']/full['Clicks_cont']
w=full['Payments_exp']/full['Clicks_exp']
full['NC'] = np.where(z<w,1,0)

full.head()

In [ ]:
GC_x=full.GC[full["GC"]==1].count()
NC_x=full.NC[full["NC"]==1].count()
n=full.NC.count()
print(f"""
No. of cases for GC: {GC_x}
No. of cases for NC: {NC_x}
No. of total cases {n}
""")

### Building a Sign Test
What we want to do after we count the amount of days in which the experiment group had a higher metric value than that of the control group, is to see if that number is likely to be seen again in a new experiment (significance). 

We assume the chance of a day like this is random (50% chance to happen) and then use the binomial distribution with  $p=0.5$ and the number of experiments (days) to tell us the probability of this happening according to a random chance.

So, according to the binomial distribution with $p=0.5$ and $n=$ total number of days; we want to know the probability of x days being a success (higher metric value in experiment). Because we are doing a two-tailed test we want to double this probability and once we have we can call it the p−value and compare it to our $\alpha$. If the p−value is greater than the $\alpha$ the result is not significant and vice-versa.

$$
p(successes) = \frac{n!}{x!(n-x)!}p^x(1-p)^{n-x}
$$

**Recall** that a p-value is the probability of observing a test statistic as or more extreme than that observed. If we observe 2 days like that, the p-value for the test is: $p = P(x\leq 2)$. Thus

$$
p(x\leq 2) = p(0)+p(1)+p(2)
$$

In [ ]:
#first a function for calculating probability of x=number of successes
def get_prob(x,n):
    p=round(mt.factorial(n)/(mt.factorial(x)*mt.factorial(n-x))*0.5**x*0.5**(n-x),4)
    return p

#next a function to compute the pvalue from probabilities of maximum x
def get_2side_pvalue(x,n):
    p=0
    for i in range(0,x+1):
        p=p+get_prob(i,n)
    return 2*p

Finally, to conduct the sign test itself: we will calculate the p-value for each metric, using the counts GC_x, NC_x and n and the function get_2side_pvalue.

In [ ]:
print ("GC Change is significant if",get_2side_pvalue(GC_x,n),"is smaller than 0.05")
print ("NC Change is significant if",get_2side_pvalue(NC_x,n),"is smaller than 0.05")

We get the same conclusions as we got from our effect size calculation: the change in Gross conversion was indeed significant, while the change in Net conversion was not.

# Conclusions & Recommendations 

At this point, once we have seen that the actual underlying goal we had was not reached (increase fraction of paying users by asking them in advance if they have the time to invest in the course), we can only recommend to not continue with change. It may have caused a change in Gross conversion, but it didn't for net conversion.